# 크롤링을 통해 받아온 데이터를 1차적으로 튜닝한 프로세스

In [214]:
import numpy as np
import pandas as pd
from IPython.display import display
import re

In [215]:
df = pd.read_csv('references.csv')

In [216]:
df.tail()

,infor,refer
1457,Why do Shareholders Allow Their Managers to be...,"1. [학술지(정기간행물)] - Asquith, Paul / 1983 / The g..."
1458,How Does Prior Information Affect Analyst Fore...,"1. [학술지(정기간행물)] - Abarbanell, J. S / 1991 / Jo..."
1459,"Studies on Information Asymmetry, Price Manipu...","1. [학술지(정기간행물)] - Avery, C / 1998 / Multidimen..."
1460,Corporate Cash Holdings and Tax-induced Debt F...,"1. [학술지(정기간행물)] - Bae, K.-H / 2002 / Tunneling..."
1461,"Corporate Diversification, Relatedness, and Fi...","1. [학술지(정기간행물)] - Amihud, Yakov / 1981 / Risk ..."


In [217]:
df['infor'].values[500]

'제품시장경쟁이 주식수익률에 미치는 영향: 한-미 FTA를 통한 증거\n류두진 | 류두원 | 황준호 | 한국재무학회 | 재무연구 | 28(3) | pp.487~512 | 2015.08 | 경영학\n피인용횟수 : 5'

In [218]:
df['refer'].values[500]

'1. [학술지(정기간행물)] - 박경서 / 2011 / 상품시장에서의 경쟁과 기업지배구조의 상호작용이 투자와 배당에 미치는 영향 / 재무연구 24 (2) : 483 ~ 522,2. [학술지(정기간행물)] - 류두원 / 2013 / 시장경쟁은 기업에 어떠한 영향을 미치는가?: 산업집중도와 기업가치 및 부채비율의 관계에 관한 실증연구 / 경영학연구 42 (2) : 435 ~ 456,3. [학술지(정기간행물)] - 류두원 / 2014 / 산업집중도에 따른 주식수익률 차이는 존재하는가? / 한국증권학회지 43 (4) : 657 ~ 678,4. [학술지(정기간행물)] - 류두원 / 2013 / 시장경쟁은 회사채 신용등급 변경공시에 따른 주식가격반응에 어떠한 영향을 주는가? / 경영학연구 42 (4) : 929 ~ 957,5. [학술지(정기간행물)] - 류두원 / 2014 / 정부기관의 중소기업 포상이 나아가야할 방향: 기업 가치에 미치는 영향을 중심으로 / 한국증권학회지 43 (1) : 47 ~ 69,6. [학술지(정기간행물)] - Ali, A. / 2009 / The Limitations of Industry Concentration Measure Constructed with Compustat Data: Implications for Finance Research / Review of Financial Studies 22 (10) : 3839 ~ 3871,7. [학술지(정기간행물)] - Alimov, A. / 2014 / Product Market Competition and the Value of Corporate Cash: Evidence from trade liberalization / Journal of Corporate Finance 25 : 122 ~ 139,8. [보고서] - Bustamante, M. C. / 2014 / Product Market Competition and Industry Returns,9. [학술지(정기간행물)] - Card, 

### 참고문헌이 나와있지 않은 논문의 수

In [219]:
len([i for i in df['refer'].values if i == '0'])

242

In [220]:
str(((len([i for i in df['refer'].values if i == '0']) / len(df)) * 100)) + '%'

'16.55266757865937%'

# Tuning_process

#### 우선 information 컬럼을 늘려준다.
- refer 컬럼을 split 하는 과정을 시행한다.

In [221]:
print(len(re.findall('[0-9]{1,}. \[',df['refer'].values[100])))
print(len(re.split('[0-9]{1,}. \[',df['refer'].values[100])))

40
41


In [222]:
head_ls = re.findall('[0-9]{1,}. \[',df['refer'].values[100])

In [223]:
testing_ls = re.split('[0-9]{1,}. \[',df['refer'].values[100])
process_ls = []
for i in range(len(testing_ls)):
    if re.search('\/ [0-9]{,4} \/',testing_ls[i]) : 
        process_ls.append(testing_ls[i])
process_ls[-2:]

['학술지(정기간행물)] - Wermers, R / 1999 / Mutual fund herding and the impact on stock prices / Journal of Finance 54 : 581 ~ 622,',
 '학술지(정기간행물)] - Yeh, Y. / 2000 / The Interaction and volatility asymmetry of unexpected returns in the greater China stock markets / Global Finance Journal 11 : 129 ~ 149']

In [224]:
title_ls = []
for i in range(len(process_ls)):
    title_ls.append(head_ls[i] + process_ls[i])
title_ls[-2:]

['39. [학술지(정기간행물)] - Wermers, R / 1999 / Mutual fund herding and the impact on stock prices / Journal of Finance 54 : 581 ~ 622,',
 '40. [학술지(정기간행물)] - Yeh, Y. / 2000 / The Interaction and volatility asymmetry of unexpected returns in the greater China stock markets / Global Finance Journal 11 : 129 ~ 149']

### information 컬럼을 늘려주는 function(generalization) 프로세스

In [225]:
#length column 을 만들어주기 위한 프로세스
final_process_ls = []

for i in range(len(df['refer'])):
    try :
        title_ls = []
        if df['refer'].values[i] != '0' : 
            head_ls = re.findall('[0-9]{1,}. \[',df['refer'].values[i])
            testing_ls = re.split('[0-9]{1,}. \[',df['refer'].values[i])
            process_ls = []
            for j in range(len(testing_ls)):
                if re.search('\/ [0-9]{,4} \/',testing_ls[j]) : 
                    process_ls.append(testing_ls[j])
            for k in range(len(process_ls)):
                title_ls.append(head_ls[k] + process_ls[k])
        else : title_ls.append('0')
        final_process_ls.append(title_ls)
    except Exception as e:
        print(i)
        print(len(process_ls),len(head_ls))
        print(e)
        break

In [226]:
df['length'] = [len(i) for i in final_process_ls]

In [227]:
new_df = pd.DataFrame()
new_df['refer'] = [j for i in final_process_ls for j in i]

In [228]:
extending_title_ls = []
for i in range(len(df)):
    for _ in range(df['length'].values[i]):
        extending_title_ls.append(df['infor'].values[i])
print(len(extending_title_ls))
new_df['infor'] = extending_title_ls

37247


In [229]:
new_df = pd.DataFrame()
new_df['infor'] = extending_title_ls
new_df['refer'] = [j for i in final_process_ls for j in i]

In [230]:
set(new_df['infor'].values) == set(df['infor'].values)
#OK
new_df.tail()

,infor,refer
37242,"Corporate Diversification, Relatedness, and Fi...","40. [기타자료] - Villalonga, Belen / 1999 / Does D..."
37243,"Corporate Diversification, Relatedness, and Fi...","41. [학술지(정기간행물)] - Villalonga, Belen / 2004 / ..."
37244,"Corporate Diversification, Relatedness, and Fi...","42. [학술지(정기간행물)] - Villalonga, Belen / 2004 / ..."
37245,"Corporate Diversification, Relatedness, and Fi...","43. [학술지(정기간행물)] - Weston, J. F / 1970 / The N..."
37246,"Corporate Diversification, Relatedness, and Fi...","44. [학술지(정기간행물)] - Whited, Toni M / 2001 / Is ..."


In [231]:
new_df.to_csv('extended_references.csv',index=False)

# Text Tuning process

# 우선 references 가 아닌 논문의 이름부터 튜닝한다.

infor 컬럼은 | 단위로 잘라줄 수 있고 \n 단위로 잘라줄 수 있다.

In [232]:
for idx in range(len(new_df['infor'].values)):
    if '\n' not in new_df['infor'].values[idx].split('|')[0]:
        print(idx)

In [233]:
new_title_ls = []
for idx in range(len(new_df['infor'].values)):
    testing_ls = new_df['infor'].values[idx].split('|')
    new_title_ls.append([j for i in [i.split('\n') for i in testing_ls] for j in i])
len(new_title_ls) == len(new_df)

True

### 튜닝한 데이터에 대해서 해당 프로세스의 새로운 데이터들을 넣어줄 새로운 데이터 프레임 tuned_df 를 만들어준다.

In [234]:
tuned_df = pd.DataFrame()
tuned_df['title_bundle'] = new_title_ls
tuned_df.tail()

,title_bundle
37242,"[Corporate Diversification, Relatedness, and F..."
37243,"[Corporate Diversification, Relatedness, and F..."
37244,"[Corporate Diversification, Relatedness, and F..."
37245,"[Corporate Diversification, Relatedness, and F..."
37246,"[Corporate Diversification, Relatedness, and F..."


### 리스트의 첫 번째 엘리먼트가 항상 논문의 이름인지 확인하는 프로세스가 필요해 보인다.

In [235]:
tuned_df['title_bundle'].values[0]

['발생액 이상현상과 차익거래기회에 관한 연구',
 '고봉찬 ',
 ' 김진우 ',
 ' 한국증권학회 ',
 ' 한국증권학회지 ',
 ' 38(1) ',
 ' pp.77~105 ',
 ' 2009.03 ',
 ' 경영학',
 '피인용횟수 : 13']

#### 이름이 넉넉잡아 5글자 이내라고 가정, 검사해본다.

In [236]:
for paper in tuned_df['title_bundle'].values:
    if len(paper[0]) <= 5 : 
        print(paper)
        
# 없으므로 naive 하지만, 각 리스트의 첫 번째 엘리먼트는 페이퍼 이름인 것을 확인할 수 있다.

### paper name 파싱! 

In [237]:
tuned_df['paper_name'] = [i[0] for i in tuned_df['title_bundle'].values]
tuned_df.tail()

,title_bundle,paper_name
37242,"[Corporate Diversification, Relatedness, and F...","Corporate Diversification, Relatedness, and Fi..."
37243,"[Corporate Diversification, Relatedness, and F...","Corporate Diversification, Relatedness, and Fi..."
37244,"[Corporate Diversification, Relatedness, and F...","Corporate Diversification, Relatedness, and Fi..."
37245,"[Corporate Diversification, Relatedness, and F...","Corporate Diversification, Relatedness, and Fi..."
37246,"[Corporate Diversification, Relatedness, and F...","Corporate Diversification, Relatedness, and Fi..."


#### 리스트에서 두 번째부터 `한국증권학회` , `한국재무관리학회` , `한국파생상품학회` , `한국재무학회` 가 나오는 엘리먼트의 리스트 인덱스 전까지는 이름 즉, 저자이다.

In [238]:
tuned_df['title_bundle'].values[0]

['발생액 이상현상과 차익거래기회에 관한 연구',
 '고봉찬 ',
 ' 김진우 ',
 ' 한국증권학회 ',
 ' 한국증권학회지 ',
 ' 38(1) ',
 ' pp.77~105 ',
 ' 2009.03 ',
 ' 경영학',
 '피인용횟수 : 13']

##### 우선 한국증권학회 , 한국재무관리학회 , 한국파생상품학회 , 한국재무학회 가 나오는 인덱스를 찾는다.

In [239]:
testing_ls = []
for paper in tuned_df['title_bundle'].values : 
    for idx in range(len(paper)):
        if re.findall('한국증권학회|한국재무관리학회|한국파생상품학회|한국재무학회',paper[idx]) : 
            testing_ls.append(idx)
            break

In [240]:
institution_ls = []
for idx in range(len(tuned_df['title_bundle'].values)):
    institution_ls.append(tuned_df['title_bundle'].values[idx][testing_ls[idx]])

In [241]:
institution_ls[-2:]

[' 한국증권학회 ', ' 한국증권학회 ']

In [242]:
for i in institution_ls : 
    if not re.findall('한국증권학회|한국재무관리학회|한국파생상품학회|한국재무학회',i):
        print(i)

# 아무것도 출력되지 않으므로 institution_ls 에는 발행기관명만 있다는 것을 알 수 있다.

In [243]:
tuned_df['institution'] = institution_ls

### 이번에는 이름을 해본다. 이름은 두번째 요소부터 발행기관명 사이의 엘리먼트이다. 발행기관명의 인덱스는 testing_ls에 넣어두었다.

In [244]:
name_ls = []
for idx,val in enumerate(tuned_df['title_bundle']):
    name_ls.append(val[1:testing_ls[idx]])
name_ls = [','.join(i) for i in name_ls]
# 리스트를 문자열 형식으로 바꿔준다.

In [245]:
tuned_df['author'] = name_ls

### 이번에는 저널명이다 testing_ls에서 1을 더해준다.

In [246]:
journal_ls = []
for idx,val in enumerate(tuned_df['title_bundle']):
    journal_ls.append(val[testing_ls[idx]+1])

In [247]:
tuned_df['journal'] = journal_ls

#### 나머지 4개의 요소를 보게 되면, 33(1)는 무엇인지 모르겠고 나머지는 각각 페이지 수와, 논문 발행 년도 , 논문의 분야 카테고리 , 마지막으로 피인용횟수가 되겠다. 각각 분배해주자.

In [248]:
tuned_df['title_bundle'].values[100]

['프로그램매매 중단장치가 주식시장의 정보비대칭에 미치는 영향',
 '박종원 ',
 ' 이우백 ',
 ' 권택호 ',
 ' 한국증권학회 ',
 ' 한국증권학회지 ',
 ' 38(3) ',
 ' pp.325~369 ',
 ' 2009.09 ',
 ' 경영학',
 '피인용횟수 : 7']

### 뭔지 모르겠지만 넣어준다.

In [249]:
dont_know_ls = []
for idx,val in enumerate(tuned_df['title_bundle']):
    dont_know_ls.append(val[-5])

tuned_df['dont_know'] = dont_know_ls

In [250]:
dont_know_ls[100]

' 38(3) '

In [251]:
[i for i in dont_know_ls if not re.findall(' [0-9]{2}\(.*\)',i)]

[]

### 페이지 수

In [252]:
page_ls = []
for idx,val in enumerate(tuned_df['title_bundle']):
    page_ls.append(val[-4])

tuned_df['page'] = page_ls

In [253]:
[i for i in page_ls if not re.findall(' pp.',i)]

[]

### 발행 시점

In [254]:
year_ls = []
for idx,val in enumerate(tuned_df['title_bundle']):
    year_ls.append(val[-3])

tuned_df['year'] = year_ls

In [255]:
[i for i in year_ls if not re.findall(' 20',i)]

[]

### 논문 분야

In [256]:
field_ls = []
for idx,val in enumerate(tuned_df['title_bundle']):
    field_ls.append(val[-2])

tuned_df['field'] = field_ls

In [257]:
[i for i in field_ls if not re.findall('경영학',i)]

[]

### 피인용 횟수

In [258]:
cited_ls = []
for idx,val in enumerate(tuned_df['title_bundle']):
    cited_ls.append(val[-1])

tuned_df['cited'] = cited_ls

In [259]:
[i for i in cited_ls if not re.findall('피인용횟수',i)]

[]

# 이렇게 해서 기존 new_df 라는 데이터 프레임에서 infor 컬럼의 튜닝을 끝냈다.

In [260]:
final_tuned_df = tuned_df.iloc[:,1:]
final_tuned_df.tail()

,paper_name,institution,author,journal,dont_know,page,year,field,cited
37242,"Corporate Diversification, Relatedness, and Fi...",한국증권학회,"배성철 , 권택호 , 이장우",Asia-Pacific Journal of Financial Studies,37(6),pp.1025~1064,2008.12,경영학,피인용횟수 : 20
37243,"Corporate Diversification, Relatedness, and Fi...",한국증권학회,"배성철 , 권택호 , 이장우",Asia-Pacific Journal of Financial Studies,37(6),pp.1025~1064,2008.12,경영학,피인용횟수 : 20
37244,"Corporate Diversification, Relatedness, and Fi...",한국증권학회,"배성철 , 권택호 , 이장우",Asia-Pacific Journal of Financial Studies,37(6),pp.1025~1064,2008.12,경영학,피인용횟수 : 20
37245,"Corporate Diversification, Relatedness, and Fi...",한국증권학회,"배성철 , 권택호 , 이장우",Asia-Pacific Journal of Financial Studies,37(6),pp.1025~1064,2008.12,경영학,피인용횟수 : 20
37246,"Corporate Diversification, Relatedness, and Fi...",한국증권학회,"배성철 , 권택호 , 이장우",Asia-Pacific Journal of Financial Studies,37(6),pp.1025~1064,2008.12,경영학,피인용횟수 : 20


In [261]:
final_tuned_df.head()

,paper_name,institution,author,journal,dont_know,page,year,field,cited
0,발생액 이상현상과 차익거래기회에 관한 연구,한국증권학회,"고봉찬 , 김진우",한국증권학회지,38(1),pp.77~105,2009.03,경영학,피인용횟수 : 13
1,발생액 이상현상과 차익거래기회에 관한 연구,한국증권학회,"고봉찬 , 김진우",한국증권학회지,38(1),pp.77~105,2009.03,경영학,피인용횟수 : 13
2,발생액 이상현상과 차익거래기회에 관한 연구,한국증권학회,"고봉찬 , 김진우",한국증권학회지,38(1),pp.77~105,2009.03,경영학,피인용횟수 : 13
3,발생액 이상현상과 차익거래기회에 관한 연구,한국증권학회,"고봉찬 , 김진우",한국증권학회지,38(1),pp.77~105,2009.03,경영학,피인용횟수 : 13
4,발생액 이상현상과 차익거래기회에 관한 연구,한국증권학회,"고봉찬 , 김진우",한국증권학회지,38(1),pp.77~105,2009.03,경영학,피인용횟수 : 13


In [262]:
final_tuned_df.to_csv('tuned_dataframe_infor.csv',index=False)

# 이번엔 refer 컬럼에 대한 튜닝을 해보자.

우선 / 로 split 이 가능해 보인다.

#### 참고문헌이 없어서 '0' 으로 등록되지 않은 리스트 중 첫 번째가   "숫자. [학술지 형태] - "  가 아닌 경우를 찾아본다.

In [263]:
new_df.tail()

,infor,refer
37242,"Corporate Diversification, Relatedness, and Fi...","40. [기타자료] - Villalonga, Belen / 1999 / Does D..."
37243,"Corporate Diversification, Relatedness, and Fi...","41. [학술지(정기간행물)] - Villalonga, Belen / 2004 / ..."
37244,"Corporate Diversification, Relatedness, and Fi...","42. [학술지(정기간행물)] - Villalonga, Belen / 2004 / ..."
37245,"Corporate Diversification, Relatedness, and Fi...","43. [학술지(정기간행물)] - Weston, J. F / 1970 / The N..."
37246,"Corporate Diversification, Relatedness, and Fi...","44. [학술지(정기간행물)] - Whited, Toni M / 2001 / Is ..."


In [264]:
new_refer_ls = []

for idx in range(len(new_df['refer'].values)):
    new_refer_ls.append(new_df['refer'].values[idx].split(' / '))
len(new_refer_ls) == len(new_df)

True

In [265]:
new_df['tuned_refer'] = new_refer_ls
new_df.tail()

,infor,refer,tuned_refer
37242,"Corporate Diversification, Relatedness, and Fi...","40. [기타자료] - Villalonga, Belen / 1999 / Does D...","[40. [기타자료] - Villalonga, Belen, 1999, Does Di..."
37243,"Corporate Diversification, Relatedness, and Fi...","41. [학술지(정기간행물)] - Villalonga, Belen / 2004 / ...","[41. [학술지(정기간행물)] - Villalonga, Belen, 2004, D..."
37244,"Corporate Diversification, Relatedness, and Fi...","42. [학술지(정기간행물)] - Villalonga, Belen / 2004 / ...","[42. [학술지(정기간행물)] - Villalonga, Belen, 2004, D..."
37245,"Corporate Diversification, Relatedness, and Fi...","43. [학술지(정기간행물)] - Weston, J. F / 1970 / The N...","[43. [학술지(정기간행물)] - Weston, J. F, 1970, The Na..."
37246,"Corporate Diversification, Relatedness, and Fi...","44. [학술지(정기간행물)] - Whited, Toni M / 2001 / Is ...","[44. [학술지(정기간행물)] - Whited, Toni M, 2001, Is I..."


In [266]:
new_df['tuned_refer'][0]

['1. [학술지(정기간행물)] - 고봉찬',
 '2007',
 '발생액 이상현상에 대한 위험평가',
 'Asia-Pacific Journal of Financial Studies 36 (3) : 425 ~ 461,']

In [267]:
set([len(i) for idx,i in enumerate(new_df['tuned_refer'])])


{1, 3, 4, 5}

In [268]:
for i in new_refer_ls : 
    if i[0] != '0' : 
        if not re.findall('[0-9]{1,}. \[.+\] -',i[0]) : 
            print(i)
            
# 없다. 참고문헌이 있는 리스트의 첫 번째 엘리먼트는 동일한 형태를 띄고 있음을 알 수 있다.

#### 참고문헌이 없어서 '0' 으로 등록되지 않은 리스트 중 두 번째가 년도의 형태가 아닌 경우를 찾아본다.

In [269]:
for i in new_refer_ls : 
    if i[0] != '0' : 
        if not re.findall('[0-9]{4}',i[1]) : 
            print(i)
            print(len(i))
            
# 안타깝게도 존재한다. 이런 경우 일단 귀속시킨 후에 교수님께 피드백을 요청한다. 7개만 나와서 그나마 다행인듯..ㅠㅠ

['27. [보고서] - Huang, J', '203', 'How much of the corporate-treasury yield spread is due to credit risk : 1 ~ 57,']
3
['6. [학술지(정기간행물)] - Bliss, R. R.', '200', 'Testing the Stability of Implied Probability Density Functions', 'Journal of Banking and Finance 26 : 381 ~ 422,']
4
['16. [학술지(정기간행물)] - Figlewski, S.', '179', 'Subjective information and market efficiency in a betting market', 'Journal of Political Economy 87 : 75 ~ 89,']
4
['11. [학술지(정기간행물)] - Ait-Sahalia, Y.', '200', 'How Often to Sample a Continuous-Time Process in the Presence of Market Microstructure Noise', 'Review of Financial Studies 18 : 351 ~ 416,']
4
['4. [학술지(정기간행물)] -', '4', 'Shareholder Preferences and Dividend Policy (45) : 993 ~ 1018,']
3
['8. [학술지(정기간행물)] -', '4', 'Two Agency Cost Explanations of Dividends (74) : 650 ~ 659,']
3
['33. [학술지(정기간행물)] - Kyle, Albert S', '53', 'Continuous auctions and insider trading', 'Econometrica 53 : 1315 ~ 1335,']
4


### 3개짜리의 구조
- 1. 논문의 성향
- 1.5. 논문의 저자(있는 경우도 있고 없는 경우도 있음)
- 2. 논문의 년도
- 3. 논문의 이름

In [270]:
print(len([i for i in new_refer_ls if len(i) == 1]),
len([i for i in new_refer_ls if len(i) == 3]),
len([i for i in new_refer_ls if len(i) == 4]),
len([i for i in new_refer_ls if len(i) == 5]))

242 8071 28840 94


### 즉, 3개짜리에는 발행기관명과 학술지명이 있는 경우, 없는 경우가 불규칙하게 분포되어 있다. 이 또한 교수님께 피드백을 받아야 하는 부분이다.

In [271]:
[i for i in new_refer_ls if len(i) == 3][0]

['53. [보고서] - New York Stock Exchange',
 '1991',
 'The Rule 80A Index Arbitrage Tick Test: Report to the U.S. Securities and Exchange Commission,']

In [272]:
[i[-1] for i in new_refer_ls if len(i) == 3][0]
# 즉, 3개짜리에는 발행기관명과 학술지명이 있는 경우, 없는 경우가 불규칙하게 분포되어 있다. 이 또한 교수님께 피드백을 받아야 하는 부분이다.

'The Rule 80A Index Arbitrage Tick Test: Report to the U.S. Securities and Exchange Commission,'

In [273]:
[j for j in [i[-1] for i in new_refer_ls if len(i) == 3] if re.findall('Journal',j)][:5]

['/ Journal of Finance 43 : 541 ~ 566,',
 'Using Financial Data to Measure Effects of Regulation Journal of Law and Economics : 121 ~ 158',
 'Theory of the Firm Journal of Financial Economics : 305 ~ 360,',
 'Efficient analytic approximation of American option values Journal of Finance : 301 ~ 320,',
 'The valuation of American put options Journal of Finance : 449 ~ 462,']

# 나중에 병합할 때 혼류가 없도록 길이에 따른 인덱스를 저장해준다

In [274]:
len_1_idx = [idx for idx,val in enumerate(new_df['tuned_refer']) if len(val) == 1] # 없는 값
len_3_idx = [idx for idx,val in enumerate(new_df['tuned_refer']) if len(val) == 3]
len_4_idx = [idx for idx,val in enumerate(new_df['tuned_refer']) if len(val) == 4]
len_5_idx = [idx for idx,val in enumerate(new_df['tuned_refer']) if len(val) == 5]
len(len_1_idx),len(len_3_idx),len(len_4_idx),len(len_5_idx)

(242, 8071, 28840, 94)

In [275]:
len(len_1_idx) +len(len_3_idx) +len(len_4_idx)  + len(len_5_idx) == len(new_df)

True

# 전체 데이터에 실시

- 기준으로 split

In [276]:
tuned_new_refer = []
for idx in range(len(new_refer_ls)):
    tuned_new_refer.append(new_refer_ls[idx][0].split(" -") + new_refer_ls[idx][1:])
len(tuned_new_refer) == len(new_refer_ls)

True

가장 첫 번째 character 엘리먼트와 3번째 엘리먼스인 년도 사이에 무엇이 있으면 하나로 병합시켜준다. (이름)

In [277]:
merge = [(1, 3)]

for ls in tuned_new_refer : 
    if ls[0] != '0': 
        if not re.findall('[0-9]{1,4}',ls[2]) : 
            for t in merge[::-1]:
                merged = ''.join(ls[t[0]:t[1]])  # merging values within a range
                ls[t[0]:t[1]] = [merged]

아무것도 안나오면 정상

In [278]:
for ls in tuned_new_refer : 
    if ls[0] != '0': 
        if not re.findall('[0-9]{1,4}',ls[2]) : 
            print(ls)

1을 제외하고 나머지는 1씩 증가했다.

In [279]:
set([len(i) for i in tuned_new_refer])

{1, 4, 5, 6}

### tuned_new_refer 와 new_refer_ls 의 차이는 단지 튜닝 전이냐 후이냐일 뿐이이여야 한다.

In [280]:
tuned_new_refer[2]

['3. [학술지(정기간행물)]',
 ' Amihud, Yakov',
 '2002',
 'Illiquidity and Stock Returns: Cross-Section and Time- Series Effects',
 'Journal of Financial Markets 5 (1) : 31 ~ 56,']

In [281]:
new_refer_ls[2]

['3. [학술지(정기간행물)] - Amihud, Yakov',
 '2002',
 'Illiquidity and Stock Returns: Cross-Section and Time- Series Effects',
 'Journal of Financial Markets 5 (1) : 31 ~ 56,']

## 길이가 4개짜리인 데이터 프레임만들기

In [282]:
[i for i in tuned_new_refer if len(i) == 4][1]

['54. [보고서]',
 ' New York Stkc Exchange',
 '1998',
 'Trading Analysis of October 27 and 28, 1997: A Report by the Division of Market Regulation to the U.S. Securities and Exchange Commission,']

## Characteristic of paper

In [283]:
char_paper_ls = [re.findall('\[.+\]',i[0])[0] for i in [i for i in tuned_new_refer if len(i) == 4]]

In [284]:
char_paper_ls[0]

'[보고서]'

In [285]:
char_paper_ls[1]

'[보고서]'

## author of paper

In [286]:
name_ls = [i[1] for i in tuned_new_refer if len(i) == 4]

In [287]:
name_ls[1]

' New York Stkc Exchange'

## year of paper published

In [288]:
year_ls = [i[2] for i in tuned_new_refer if len(i) == 4]

In [289]:
year_ls[1]

'1998'

## name of paper

In [290]:
paper_ls = [i[3] for i in tuned_new_refer if len(i) == 4]

In [291]:
paper_ls[1]

'Trading Analysis of October 27 and 28, 1997: A Report by the Division of Market Regulation to the U.S. Securities and Exchange Commission,'

In [292]:
len(char_paper_ls) == len(name_ls) == len(year_ls) == len(paper_ls)

True

In [293]:
len_4_df = pd.DataFrame()
len_4_df['char'] = char_paper_ls
len_4_df['name'] = name_ls
len_4_df['year'] = year_ls
len_4_df['paper'] = paper_ls

len_4_df.tail()

,char,name,year,paper
8066,[학술지(정기간행물)],"Plesko, G. A",2003,Journal of Accounting and Economics 35 : 201 ~...
8067,[기타자료],"Ross, M. P",1997,"Corporate hedging: What, why and how? Working ..."
8068,[학술지(정기간행물)],"Baek, Jae-Seung",2004,"Journal of Financial Economics 71 : 265 ~ 313,"
8069,[기타자료],"Chandler, A. D",1977,"The Visible Hand, Belknap Press, Cambridge, MA..."
8070,[기타자료],"Villalonga, Belen",1999,Does Diversification Cause the “Diversificatio...


이전에 길이가 3이였던 데이터 프레임이 4가 된 것이기 때문에, 인덱스를 일치시켜주기 위해서 대입해준다.

In [294]:
len_4_df.index = len_3_idx

In [295]:
len_4_df.tail()

,char,name,year,paper
37195,[학술지(정기간행물)],"Plesko, G. A",2003,Journal of Accounting and Economics 35 : 201 ~...
37196,[기타자료],"Ross, M. P",1997,"Corporate hedging: What, why and how? Working ..."
37206,[학술지(정기간행물)],"Baek, Jae-Seung",2004,"Journal of Financial Economics 71 : 265 ~ 313,"
37211,[기타자료],"Chandler, A. D",1977,"The Visible Hand, Belknap Press, Cambridge, MA..."
37242,[기타자료],"Villalonga, Belen",1999,Does Diversification Cause the “Diversificatio...


In [296]:
len_4_df.tail(1).values

array([['[기타자료]', ' Villalonga, Belen', '1999',
        'Does Diversification Cause the “Diversification Discount? Working paper, Harvard University,']],
      dtype=object)

In [297]:
new_df.iloc[37242:37243]['refer'].values

array(['40. [기타자료] - Villalonga, Belen / 1999 / Does Diversification Cause the “Diversification Discount? Working paper, Harvard University,'],
      dtype=object)

In [298]:
list(len_4_df.index.values) == len_3_idx

True

---------------

# 4개짜리의 구조
- 마지막 엘리먼트는 저널명이 되겠다.

In [299]:
[i for i in new_refer_ls if len(i) == 4][800]

['22. [학술지(정기간행물)] - Jensen, M',
 '1976',
 'Theory of the Firm: Managerial Behavior, Agency Costs and Ownership Structure',
 'Journal of Financial Economics 3 : 305 ~ 360,']

In [300]:
list(set([paper[-1] for paper in [i for i in new_refer_ls if len(i) == 4]]))[-5:]

['Journal of Banking and Finance 22 : 1481 ~ 1506,',
 'Quarterly Journal of Economics 104 : 1 ~ 24,',
 'The Journal of Finance 50 : 1175 ~ 1199,',
 'Journal of Financial and Quantitative Analysis 28 (2) : 177 ~ 194,',
 'Financial Management 18 : 36 ~ 46,']

In [301]:
print(len([paper[-1] for paper in [i for i in new_refer_ls if len(i) == 4]]))

28840


- 아래는 길이가 4개인 리스트에서 저널명을 나타내는 마지막 리스트를 빼온 것이다.
- 이 리스트를 통해서 길이가 3개짜리인 리스트의 마지막에서 저널명이 포함된 것을 빼와본다.
- 우선 일반화를 위해서 숫자를 다 빼고 유니크한 값으로 변환시켜준다.

In [302]:
[paper[-1] for paper in [i for i in new_refer_ls if len(i) == 4]][-5:]

['Journal of Financial Economics 26 : 3 ~ 27,',
 'Journal of Finance 59 : 479 ~ 506,',
 'Financial Management 33 : 5 ~ 27,',
 'St. John’s Review 44 : 66 ~ 80,',
 'Journal of Finance 56 : 1667 ~ 1691']

In [303]:
len([paper[-1] for paper in [i for i in new_refer_ls if len(i) == 4]])

28840

# 4개짜리 데이터 프레임으로 만들기

In [304]:
[i for i in tuned_new_refer if len(i) == 5][0]

['1. [학술지(정기간행물)]',
 ' 고봉찬',
 '2007',
 '발생액 이상현상에 대한 위험평가',
 'Asia-Pacific Journal of Financial Studies 36 (3) : 425 ~ 461,']

## Characteristic of paper

In [305]:
char_paper_ls = [re.findall('\[.+\]',i[0])[0] for i in [i for i in tuned_new_refer if len(i) == 5]]

In [306]:
char_paper_ls[0]

'[학술지(정기간행물)]'

## author of paper

In [307]:
name_ls = [i[1] for i in tuned_new_refer if len(i) == 5]

In [308]:
name_ls[0]

' 고봉찬'

## year of paper published

In [309]:
year_ls = [i[2] for i in tuned_new_refer if len(i) == 5]

In [310]:
year_ls[0]

'2007'

## name of paper

In [311]:
paper_ls = [i[3] for i in tuned_new_refer if len(i) == 5]

In [312]:
paper_ls[0]

'발생액 이상현상에 대한 위험평가'

## journal name which paper published

In [313]:
journal_ls = [i[-1] for i in tuned_new_refer if len(i) == 5]

In [314]:
journal_ls[0]

'Asia-Pacific Journal of Financial Studies 36 (3) : 425 ~ 461,'

In [315]:
len(char_paper_ls) == len(name_ls) == len(year_ls) == len(paper_ls) == len(journal_ls)

True

In [316]:
len_5_df = pd.DataFrame()
len_5_df['char'] = char_paper_ls
len_5_df['name'] = name_ls
len_5_df['year'] = year_ls
len_5_df['paper'] = paper_ls
len_5_df['journal'] = journal_ls

len_5_df.tail()

,char,name,year,paper,journal
28835,[학술지(정기간행물)],"Stulz, Ren?M",1990,Managerial Direction and Optimal Financing Pol...,"Journal of Financial Economics 26 : 3 ~ 27,"
28836,[학술지(정기간행물)],"Villalonga, Belen",2004,Diversification discount or premium? New Evide...,"Journal of Finance 59 : 479 ~ 506,"
28837,[학술지(정기간행물)],"Villalonga, Belen",2004,Does Diversification Cause the Diversification...,"Financial Management 33 : 5 ~ 27,"
28838,[학술지(정기간행물)],"Weston, J. F",1970,The Nature and Significance of Conglomerate Firms,"St. John’s Review 44 : 66 ~ 80,"
28839,[학술지(정기간행물)],"Whited, Toni M",2001,Is It Inefficient Investment That Causes the D...,Journal of Finance 56 : 1667 ~ 1691


In [317]:
len_5_df.index = len_4_idx

In [318]:
len_5_df.tail()

,char,name,year,paper,journal
37241,[학술지(정기간행물)],"Stulz, Ren?M",1990,Managerial Direction and Optimal Financing Pol...,"Journal of Financial Economics 26 : 3 ~ 27,"
37243,[학술지(정기간행물)],"Villalonga, Belen",2004,Diversification discount or premium? New Evide...,"Journal of Finance 59 : 479 ~ 506,"
37244,[학술지(정기간행물)],"Villalonga, Belen",2004,Does Diversification Cause the Diversification...,"Financial Management 33 : 5 ~ 27,"
37245,[학술지(정기간행물)],"Weston, J. F",1970,The Nature and Significance of Conglomerate Firms,"St. John’s Review 44 : 66 ~ 80,"
37246,[학술지(정기간행물)],"Whited, Toni M",2001,Is It Inefficient Investment That Causes the D...,Journal of Finance 56 : 1667 ~ 1691


In [319]:
list(len_5_df.index.values) == len_4_idx

True

In [320]:
len_5_df.tail(1).values

array([['[학술지(정기간행물)]', ' Whited, Toni M', '2001',
        'Is It Inefficient Investment That Causes the Diversification Discount',
        'Journal of Finance 56 : 1667 ~ 1691']], dtype=object)

In [321]:
new_df['tuned_refer'][37246]

['44. [학술지(정기간행물)] - Whited, Toni M',
 '2001',
 'Is It Inefficient Investment That Causes the Diversification Discount',
 'Journal of Finance 56 : 1667 ~ 1691']

-------------------------------------

# 5개짜리의 구조

In [322]:
len([i for i in new_refer_ls if len(i) == 5])

94

- 우선 첫 번째 리스트에서 - 이후에 성명이 나와있는 경우가 있다.
- 마지막과 그 전 리스트 엘리먼트는 해당 저자의 소속과 학위가 나와있다.

# 학위

In [323]:
set([paper[-2] for paper in [i for i in new_refer_ls if len(i) == 5]])

{'Ph.D', '博士', '박사', '석사'}

# 소속

In [324]:
list(set([paper[-1] for paper in [i for i in new_refer_ls if len(i) == 5]]))[-5:]

['한국외국어대학교,',
 '아주대학교 공공정책대학원,',
 'KAIST',
 'University of Southern California',
 '연세대학교 경제대학원,']

In [325]:
len_5_df.index

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            37236, 37237, 37238, 37239, 37240, 37241, 37243, 37244, 37245,
            37246],
           dtype='int64', length=28840)

# 길이가 6개인 리스트를 나눠서 데이터 프레임으로 넣기

In [326]:
[i for i in tuned_new_refer if len(i) == 6][0]

['10. [학위논문]',
 ' 張榮光',
 '1985',
 '企業合倂과 株式取得이 株主의 富에 미치는 影響에 關한 實證的 硏究',
 '박사',
 '고려대학교 대학원,']

## Characteristic of paper

In [327]:
char_paper_ls = [re.findall('\[.+\]',i[0])[0] for i in [i for i in tuned_new_refer if len(i) == 6]]

## Author of paper

In [328]:
name_ls = []

for k in range(len([i for i in tuned_new_refer if len(i) == 6])):
    name_ls.append([i for i in tuned_new_refer if len(i) == 6][k]\
[1:[idx for idx,val in enumerate([i for i in tuned_new_refer if len(i) == 6][k]) if re.findall("[0-9]{4}",val)][0]][0])

## Year of paper published

In [329]:
year_ls = []

for k in range(len([i for i in tuned_new_refer if len(i) == 6])):
    year_ls.append([i for i in tuned_new_refer if len(i) == 6][k]\
[[idx for idx,val in enumerate([i for i in tuned_new_refer if len(i) == 6][k]) if re.findall("[0-9]{4}",val)][0]])

## Name of paper

In [330]:
paper_ls = []

for k in range(len([i for i in tuned_new_refer if len(i) == 6])):
    paper_ls.append([i for i in tuned_new_refer if len(i) == 6][k]\
[[idx+1 for idx,val in enumerate([i for i in tuned_new_refer if len(i) == 6][k]) if re.findall("[0-9]{4}",val)][0]])

## Details about Author (degree , school)

In [331]:
degree_ls = [i[-2] for i in tuned_new_refer if len(i) == 6]
school_ls = [i[-1] for i in tuned_new_refer if len(i) == 6]

In [332]:
len(char_paper_ls) == len(name_ls) == len(year_ls) == len(paper_ls) == len(degree_ls) == len(school_ls)

True

In [333]:
len_6_df = pd.DataFrame()
len_6_df['char'] = char_paper_ls
len_6_df['name'] = name_ls
len_6_df['year'] = year_ls
len_6_df['paper'] = paper_ls
len_6_df['degree'] = degree_ls
len_6_df['school'] = school_ls

len_6_df.tail()

,char,name,year,paper,degree,school
89,[학위논문],김동현,2008,국내 민간의료보험 시장의 역선택에 관한 실증연구,석사,"경희대학교 대학원,"
90,[학위논문],이수진,2012,민간의료보험이 의료수요에 미치는 영향 : 도덕적 해이와 역선택 효과의 실증적 구분,석사,"연세대학교 대학원,"
91,[학위논문],한영하,2014,한국주택금융공사 보금자리론 조기상환율 추정 및 가치산정 연구,박사,"연세대학교 대학원,"
92,[학위논문],"Low, A",1992,Essays on asymmetric information in internatio...,박사,"University of California,"
93,[학위논문],"Averbukh, V",1997,Pricing American options using Monte Carlo sim...,박사,"Cornell University,"


In [334]:
len_6_df.index = len_5_idx

In [335]:
len_6_df.tail()

,char,name,year,paper,degree,school
32447,[학위논문],김동현,2008,국내 민간의료보험 시장의 역선택에 관한 실증연구,석사,"경희대학교 대학원,"
32453,[학위논문],이수진,2012,민간의료보험이 의료수요에 미치는 영향 : 도덕적 해이와 역선택 효과의 실증적 구분,석사,"연세대학교 대학원,"
32878,[학위논문],한영하,2014,한국주택금융공사 보금자리론 조기상환율 추정 및 가치산정 연구,박사,"연세대학교 대학원,"
36364,[학위논문],"Low, A",1992,Essays on asymmetric information in internatio...,박사,"University of California,"
36602,[학위논문],"Averbukh, V",1997,Pricing American options using Monte Carlo sim...,박사,"Cornell University,"


In [336]:
list(len_6_df.index) == len_5_idx

True

In [337]:
new_df['tuned_refer'][36602]

['1. [학위논문] - Averbukh, V',
 '1997',
 'Pricing American options using Monte Carlo simulation',
 '박사',
 'Cornell University,']

## 결측치만 모여있는 데이터프레임 만들기

In [338]:
len(len_1_idx)

242

In [339]:
len_1_df = pd.DataFrame(index=len_1_idx,columns=['char','name','year','paper','degree','school'])
len_1_df.tail()

,char,name,year,paper,degree,school
33185,NaN,NaN,NaN,NaN,NaN,NaN
33186,NaN,NaN,NaN,NaN,NaN,NaN
33187,NaN,NaN,NaN,NaN,NaN,NaN
35176,NaN,NaN,NaN,NaN,NaN,NaN
35226,NaN,NaN,NaN,NaN,NaN,NaN


In [340]:
new_df.iloc[35226]

infor          한국주식시장의 이익정보 불확실성위험과 1월효과\n김동철 | 신성호 | 한국증권학회 ...
refer                                                          0
tuned_refer                                                  [0]
Name: 35226, dtype: object


# concatenate the 3 dataframe

In [341]:
display(len_1_df.tail())
display(len_4_df.tail())
display(len_5_df.tail())
display(len_6_df.tail())

,char,name,year,paper,degree,school
33185,NaN,NaN,NaN,NaN,NaN,NaN
33186,NaN,NaN,NaN,NaN,NaN,NaN
33187,NaN,NaN,NaN,NaN,NaN,NaN
35176,NaN,NaN,NaN,NaN,NaN,NaN
35226,NaN,NaN,NaN,NaN,NaN,NaN


,char,name,year,paper
37195,[학술지(정기간행물)],"Plesko, G. A",2003,Journal of Accounting and Economics 35 : 201 ~...
37196,[기타자료],"Ross, M. P",1997,"Corporate hedging: What, why and how? Working ..."
37206,[학술지(정기간행물)],"Baek, Jae-Seung",2004,"Journal of Financial Economics 71 : 265 ~ 313,"
37211,[기타자료],"Chandler, A. D",1977,"The Visible Hand, Belknap Press, Cambridge, MA..."
37242,[기타자료],"Villalonga, Belen",1999,Does Diversification Cause the “Diversificatio...


,char,name,year,paper,journal
37241,[학술지(정기간행물)],"Stulz, Ren?M",1990,Managerial Direction and Optimal Financing Pol...,"Journal of Financial Economics 26 : 3 ~ 27,"
37243,[학술지(정기간행물)],"Villalonga, Belen",2004,Diversification discount or premium? New Evide...,"Journal of Finance 59 : 479 ~ 506,"
37244,[학술지(정기간행물)],"Villalonga, Belen",2004,Does Diversification Cause the Diversification...,"Financial Management 33 : 5 ~ 27,"
37245,[학술지(정기간행물)],"Weston, J. F",1970,The Nature and Significance of Conglomerate Firms,"St. John’s Review 44 : 66 ~ 80,"
37246,[학술지(정기간행물)],"Whited, Toni M",2001,Is It Inefficient Investment That Causes the D...,Journal of Finance 56 : 1667 ~ 1691


,char,name,year,paper,degree,school
32447,[학위논문],김동현,2008,국내 민간의료보험 시장의 역선택에 관한 실증연구,석사,"경희대학교 대학원,"
32453,[학위논문],이수진,2012,민간의료보험이 의료수요에 미치는 영향 : 도덕적 해이와 역선택 효과의 실증적 구분,석사,"연세대학교 대학원,"
32878,[학위논문],한영하,2014,한국주택금융공사 보금자리론 조기상환율 추정 및 가치산정 연구,박사,"연세대학교 대학원,"
36364,[학위논문],"Low, A",1992,Essays on asymmetric information in internatio...,박사,"University of California,"
36602,[학위논문],"Averbukh, V",1997,Pricing American options using Monte Carlo sim...,박사,"Cornell University,"


In [342]:
final_df = pd.concat([len_4_df,len_5_df],axis=0)
final_df = pd.concat([final_df,len_6_df],axis=0)
final_df = pd.concat([final_df,len_1_df],axis=0)
# final_df.reset_index(drop=True,inplace=True)
final_df.tail()

,char,degree,journal,name,paper,school,year
33185,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33186,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33187,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35176,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35226,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [343]:
list(final_df.index.values) == len_3_idx + len_4_idx + len_5_idx + len_1_idx

True

In [344]:
testing_df = final_df.reset_index()
testing_df.sort_values(by='index',inplace=True)
final_df = testing_df.set_index('index')

In [345]:
final_df.to_csv('tuned_dataframe_refer.csv',index=False)

In [346]:
final_df.shape

(37247, 7)

In [347]:
final_df.tail()

,char,degree,journal,name,paper,school,year
index,,,,,,,
37242,[기타자료],NaN,NaN,"Villalonga, Belen",Does Diversification Cause the “Diversificatio...,NaN,1999
37243,[학술지(정기간행물)],NaN,"Journal of Finance 59 : 479 ~ 506,","Villalonga, Belen",Diversification discount or premium? New Evide...,NaN,2004
37244,[학술지(정기간행물)],NaN,"Financial Management 33 : 5 ~ 27,","Villalonga, Belen",Does Diversification Cause the Diversification...,NaN,2004
37245,[학술지(정기간행물)],NaN,"St. John’s Review 44 : 66 ~ 80,","Weston, J. F",The Nature and Significance of Conglomerate Firms,NaN,1970
37246,[학술지(정기간행물)],NaN,Journal of Finance 56 : 1667 ~ 1691,"Whited, Toni M",Is It Inefficient Investment That Causes the D...,NaN,2001


In [348]:
new_df.tail()

,infor,refer,tuned_refer
37242,"Corporate Diversification, Relatedness, and Fi...","40. [기타자료] - Villalonga, Belen / 1999 / Does D...","[40. [기타자료] - Villalonga, Belen, 1999, Does Di..."
37243,"Corporate Diversification, Relatedness, and Fi...","41. [학술지(정기간행물)] - Villalonga, Belen / 2004 / ...","[41. [학술지(정기간행물)] - Villalonga, Belen, 2004, D..."
37244,"Corporate Diversification, Relatedness, and Fi...","42. [학술지(정기간행물)] - Villalonga, Belen / 2004 / ...","[42. [학술지(정기간행물)] - Villalonga, Belen, 2004, D..."
37245,"Corporate Diversification, Relatedness, and Fi...","43. [학술지(정기간행물)] - Weston, J. F / 1970 / The N...","[43. [학술지(정기간행물)] - Weston, J. F, 1970, The Na..."
37246,"Corporate Diversification, Relatedness, and Fi...","44. [학술지(정기간행물)] - Whited, Toni M / 2001 / Is ...","[44. [학술지(정기간행물)] - Whited, Toni M, 2001, Is I..."


# concatenate tuned_infor and tuned_refer dataframe

In [349]:
infor_csv = pd.read_csv("tuned_dataframe_infor.csv")
refer_csv = pd.read_csv("tuned_dataframe_refer.csv")

df = pd.concat([infor_csv,refer_csv],axis=1)

In [350]:
df.columns = ['paper_name_infor', 'institution_infor', 'author_infor', 'journal_infor', 'dont_know_infor', 'page_infor',
       'year_infor', 'field_infor', 'cited_infor', 'char_refer', 'degree_refer', 'journal_refer', 'name_refer', 'paper_refer',
       'school_refer', 'year_refer']

In [351]:
df.to_csv('tuned_dataframe.csv',index=False)

In [352]:
df.tail()

,paper_name_infor,institution_infor,author_infor,journal_infor,dont_know_infor,page_infor,year_infor,field_infor,cited_infor,char_refer,degree_refer,journal_refer,name_refer,paper_refer,school_refer,year_refer
37242,"Corporate Diversification, Relatedness, and Fi...",한국증권학회,"배성철 , 권택호 , 이장우",Asia-Pacific Journal of Financial Studies,37(6),pp.1025~1064,2008.12,경영학,피인용횟수 : 20,[기타자료],NaN,NaN,"Villalonga, Belen",Does Diversification Cause the “Diversificatio...,NaN,1999.0
37243,"Corporate Diversification, Relatedness, and Fi...",한국증권학회,"배성철 , 권택호 , 이장우",Asia-Pacific Journal of Financial Studies,37(6),pp.1025~1064,2008.12,경영학,피인용횟수 : 20,[학술지(정기간행물)],NaN,"Journal of Finance 59 : 479 ~ 506,","Villalonga, Belen",Diversification discount or premium? New Evide...,NaN,2004.0
37244,"Corporate Diversification, Relatedness, and Fi...",한국증권학회,"배성철 , 권택호 , 이장우",Asia-Pacific Journal of Financial Studies,37(6),pp.1025~1064,2008.12,경영학,피인용횟수 : 20,[학술지(정기간행물)],NaN,"Financial Management 33 : 5 ~ 27,","Villalonga, Belen",Does Diversification Cause the Diversification...,NaN,2004.0
37245,"Corporate Diversification, Relatedness, and Fi...",한국증권학회,"배성철 , 권택호 , 이장우",Asia-Pacific Journal of Financial Studies,37(6),pp.1025~1064,2008.12,경영학,피인용횟수 : 20,[학술지(정기간행물)],NaN,"St. John’s Review 44 : 66 ~ 80,","Weston, J. F",The Nature and Significance of Conglomerate Firms,NaN,1970.0
37246,"Corporate Diversification, Relatedness, and Fi...",한국증권학회,"배성철 , 권택호 , 이장우",Asia-Pacific Journal of Financial Studies,37(6),pp.1025~1064,2008.12,경영학,피인용횟수 : 20,[학술지(정기간행물)],NaN,Journal of Finance 56 : 1667 ~ 1691,"Whited, Toni M",Is It Inefficient Investment That Causes the D...,NaN,2001.0


## 정리
- 4개짜리 리스트 (8071개)
    - 논문의 특성
    - 이름(공백이라고 해도 무방)
    - 년도
    - 논문 이름(저널이름이 함께 포함되어 있는 경우가 있음, Journal만 있는 경우가 900개 넘는다.)
- 5개짜리 리스트 (28840개)
    - 논문의 특성
    - 이름
    - 년도
    - 논문의 이름
    - 저널의 이름
- 6개짜리 리스트 (94개)
    - 논문의 특성
    - 이름
    - 년도
    - 논문의 이름
    - 저자의 학위
    - 저자의 소속

In [353]:
(len([i for i in tuned_new_refer if len(i) == 4]),
len([i for i in tuned_new_refer if len(i) == 5]),
len([i for i in tuned_new_refer if len(i) == 6]))

(8071, 28840, 94)

In [354]:
[i for i in tuned_new_refer if len(i) == 4][0]

['53. [보고서]',
 ' New York Stock Exchange',
 '1991',
 'The Rule 80A Index Arbitrage Tick Test: Report to the U.S. Securities and Exchange Commission,']

In [355]:
[i for i in tuned_new_refer if len(i) == 5][0]

['1. [학술지(정기간행물)]',
 ' 고봉찬',
 '2007',
 '발생액 이상현상에 대한 위험평가',
 'Asia-Pacific Journal of Financial Studies 36 (3) : 425 ~ 461,']

In [356]:
[i for i in tuned_new_refer if len(i) == 6][0]

['10. [학위논문]',
 ' 張榮光',
 '1985',
 '企業合倂과 株式取得이 株主의 富에 미치는 影響에 關한 實證的 硏究',
 '박사',
 '고려대학교 대학원,']

# ! 34,50,52 부분을 교수님께 여쭈어보아야 한다.

- link
- raw data
- character

- cited = 0 
- 2007년부터 신뢰성
- 연도별 , 저널별 , 한 사이팅 페이터가 몇 개의 사이티드 페이
- 시간별로 연도별 , 저널별 평균값을 취해서 신뢰성을 체크